# Caso Práctico: Clasificación de sentimientos

## Conjunto de datos

Este es un conjunto de datos está formado por 25.000 críticas de películas del IMDB (Internet Movie Database), etiquetadas por sentimiento (positivo/negativo). Las reseñas han sido preprocesadas, y cada reseña está codificada como una lista de índices de palabras (números enteros). Para mayor comodidad, las palabras se indexan por frecuencia general en el conjunto de datos, de modo que, por ejemplo, el número entero "3" codifica la tercera palabra más frecuente en los datos. Esto permite realizar operaciones de filtrado rápido como considerar sólo las 10.000 palabras más comunes.

## 1. Cargando el conjunto de datos

In [ ]:
from tensorflow.keras.datasets import imdb

In [ ]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

El argumento _num_words_ indica que únicamente tendremos en cuenta las 10.000 palabras más frecuentes del conjunto de datos. Por lo tanto, el resto de palabras que sean poco frecuentes, serán descartadas y no formarán parte de las reseñas con las que entrenaremos y probaremos el algoritmo.

Tanto _train_data_ como _test_data_ se corresponden con una lista de reseñas. Cada una de las reseñas se corresponde con una lista de números enteros que representan el índice de una palabra. Para conocer que índice tiene cada palabra, podemos llamar al método _get_word_index()_

In [ ]:
# Índices de las palabras ordenados por frecuencias
word_index = imdb.get_word_index()
word_index

In [ ]:
# Primera reseña del conjunto de datos de entrenamiento
print(train_data[0])

Es importante tener en cuenta que a la hora de visualizar las reseñas debemos aplicar un offset de -3 debido a que el número 0, 1 y 2 están reservados para "padding", "start of sequence" y "unknown" respectivamente.

In [ ]:
# Decodificando la reseña utilizando los índices de las palabras
reversed_word_index = {v: k for k, v in word_index.items()}
' '.join([reversed_word_index.get(i - 3, "({})".format(i)) for i in train_data[0]])

Las variables _train_labels_ y _test_labels_ se corresponden con una lista de valores que puede ser 1, si la reseña es positiva, o 0, si la reseña es negativa

In [ ]:
# La reseña anterior se corresponde con una reseña positiva
print(train_labels[0])

## 2. Preprocesamiento del conjunto de datos

A pesar de que ya tenemos codificadas las reseñas para que esten representadas por valores numéricos en lugar de por cadenas de texto, existen algunos factores que no permiten que sean conjuntos de datos apropiados para proporcionarle a una red neuronal artifical.

Una de las limitaciones fundamentales es que cada reseña tiene un número diferente de palabras.

In [ ]:
import numpy as np
# las entradas de las RN necesitan un tamaño fijo y las reseñas tienes un nº de palabras variable -> relleno a 0
# Vectorizamos las reseñas convirtiendolas en vectores de 10.000 valores 1s y 0s
def vectorize(seqs, dim=10000): # 10.000 ceros
    results = np.zeros((len(seqs), dim))
    for i, seq in enumerate(seqs): # si en esa posición de la reseña hay una palabra, pongo un 1.
        results[i, seq] = 1.
    return results

In [ ]:
# Un ejemplo con 15 -> Para que lo anterior funcione se basa en la siguiente propiedad
a = np.zeros(15)
print("Array original:\t\t", a)
a[[0, 3, 5, 7, 9]] = 1.
print("Array modificado:\t", a)

Vectorizamos tanto el conjunto de datos de entrenamiento, como el conjunto de datos de pruebas.

In [ ]:
# Vectorizamos el conjunto de datos de entrenamiento y el de pruebas
X_train = vectorize(train_data, 10000)
X_test = vectorize(test_data, 10000)

In [ ]:
# Cada una de las reseñas se corresponde con un vector formado por 1s y 0s
print("Valores originales:\t", train_data[0][:15])
print("Valores vectorizados:\t", X_train[0][:15])

## 3. División del conjunto de datos

In [ ]:
from sklearn.model_selection import train_test_split
X_test, X_val, Y_test, Y_val = train_test_split(X_test, test_labels, test_size=0.5)

In [ ]:
Y_train = train_labels

In [ ]:
print("Longitud subconjunto entrenamiento: ", len(X_train))
print("Longitud subconjunto validación: ", len(X_val))
print("Longitud subconjunto pruebas: ", len(X_test))

## 4. Construcción de la red neuronal artificial

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
model = models.Sequential() # modelo secuencial de Keros y no el modelo funcional
model.add(layers.Dense(16, activation='tanh', input_shape=(10000,)))
model.add(layers.Dense(16, activation='tanh'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
from tensorflow.keras import optimizers
model.compile(
    optimizer=optimizers.SGD(learning_rate=0.001), 
    loss='binary_crossentropy',
    metrics=['accuracy', 'Precision']
)

In [ ]:
history = model.fit(
    X_train,
    Y_train,
    epochs=40,
    validation_data=(X_val, Y_val))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history.history)[['loss', 'val_loss']].plot(figsize=(10, 6))
plt.grid(True)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot(figsize=(10, 6))
plt.grid(True)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()

### 4.1 Evaluación con el subconjunto de pruebas

In [ ]:
test_loss, test_acc, test_prec = model.evaluate(X_test, Y_test)

In [ ]:
print('test_acc:', test_acc)
print('test_prec:', test_prec)

## 5. Otros preprocesamientos del conjunto de datos

### Cargando el conjunto de datos

In [ ]:
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
# usaré los índices de las palabras en lugar de las palabras
word_index = imdb.get_word_index()

In [ ]:
# Decodificando la reseña utilizando los índices de las palabras de manera más eficientes
reversed_word_index = {v: k for k, v in word_index.items()}
" ".join([reversed_word_index.get(i - 3, "({})".format(i)) for i in train_data[0]])

In [ ]:
train_data[0]

### Preparación del conjunto de datos

In [ ]:
X_train_prep = []
X_test_prep = []

In [ ]:
# Preprocesando el conjunto de datos de entrenamiento
for review in train_data:
    X_train_prep.append(
        " ".join([reversed_word_index.get(i - 3, "") for i in review]))

In [ ]:
X_train_prep[0]  #son frases con palabras, pero necesitaré usar índices

In [ ]:
# Preprocesando el conjunto de datos pruebas
for review in test_data:
    X_test_prep.append(
        " ".join([reversed_word_index.get(i - 3, "") for i in review]))

In [ ]:
X_test_prep[0] #son frases con palabras, pero necesitaré usar índices

In [ ]:
Y_train = train_labels
Y_test = test_labels

### División del conjunto de datos

Lo ideal es aplicar la división del conjunto de datos antes de realizar ningún preprocesamiento para que no se infiera ningún tipo de tendencia como consecuencia de la transformación.

In [ ]:
from sklearn.model_selection import train_test_split
X_test_prep, X_val_prep, Y_test, Y_val = train_test_split(X_test_prep, Y_test, test_size=0.5)

In [ ]:
print("Longitud subconjunto entrenamiento: ", len(X_train_prep)) # frases
print("Longitud subconjunto validación: ", len(X_val_prep)) # frases
print("Longitud subconjunto pruebas: ", len(X_test_prep)) # frases

### Vectorización del conjunto de datos

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
# .fit() se usa para entrenar
X_train = vectorizer.fit_transform(X_train_prep) # fit y transform para Datos de Entrenamiento

In [ ]:
X_train = X_train.toarray() # convierto el vector transformado en un arrya de numpy

In [ ]:
import pandas as pd
pd.DataFrame(X_train, columns=[vectorizer.get_feature_names()]) # .get_feature_names() -> coge las features (carecterísticas) de entrada a al red neuronal y las pone como nombre de las columnas

In [ ]:
X_test = vectorizer.transform(X_test_prep) # solo transform(sin .fit) para Datos de Test
X_test = X_test.toarray()

In [ ]:
X_val = vectorizer.transform(X_val_prep) # solo transform(sin .fit) para Datos de Validación
X_val = X_val.toarray()

### Entrenamiento de la red neuronal artificial

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
model = models.Sequential() # Modelo Keras secuencial
model.add(layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
from tensorflow.keras import optimizers
model.compile(
    optimizer=optimizers.SGD(learning_rate=0.0005), # 'sdg' es 1
    loss='binary_crossentropy',
    metrics=['accuracy', 'Precision']
)

In [ ]:
history = model.fit( # se entrena la Red Neuronal con .compile y .fit
    X_train,
    Y_train,
    epochs=10,
    validation_data=(X_val, Y_val))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history.history)[['loss', 'val_loss']].plot(figsize=(10, 6))
plt.grid(True)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot(figsize=(10, 6))
plt.grid(True)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()

### Evaluación con el subconjunto de pruebas

In [ ]:
test_loss, test_acc, test_prec = model.evaluate(X_test, Y_test)

In [ ]:
print('test_acc:', test_acc)
print('test_prec:', test_prec)